참고문헌: PyTorch로 시작하는 딥러닝 입문 (WikiDocs), 파이썬 딥러닝 파이토치 (이경택,방성수, 안상준 지음), 정보문화사

# 미니 배치와 데이터 로드 (Mini Batch and Data Load)

이번 챕터에서 배우는 내용은 선형 회귀에 한정되는 내용은 아닙니다. 이번 챕터에서는 데이터를 로드하는 방법과 미니 배치 경사 하강법(Minibatch Gradient Descent)에 대해서 학습합니다.

# 1. 미니 배치와 배치 크기(Mini Batch and Batch Size)

앞서 배운 다중 선형 회귀에서 사용했던 데이터를 상기해봅시다.


```
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
```

위 데이터의 샘플의 개수는 5개입니다. 전체 데이터를 하나의 행렬로 선언하여 전체 데이터에 대해서 경사 하강법을 수행하여 학습할 수 있습니다. 그런데 위 데이터는 현업에서 다루게 되는 방대한 양의 데이터에 비하면 굉장히 적은 양입니다. 만약, 데이터가 수십만개 이상이라면 전체 데이터에 대해서 경사 하강법을 수행하는 것은 매우 느릴 뿐만 아니라 많은 계산량이 필요합니다. 정말 어쩌면 메모리의 한계로 계산이 불가능한 경우도 있을 수 있습니다.

그렇기 때문에 전체 데이터를 더 작은 단위로 나누어서 해당 단위로 학습하는 개념이 나오게 되었습니다.
이 단위를 미니 배치(Mini Batch)라고 합니다.

미니 배치 학습에서는 미니 배치의 개수만큼 경사 하강법을 수행해야 전체 데이터가 한 번 전부 사용되어 1 에포크(Epoch)가 됩니다. 미니 배치의 개수는 결국 미니 배치의 크기를 몇으로 하느냐에 따라서 달라지는데 미니 배치의 크기를 배치 크기(batch size)라고 합니다.

* 전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법을 **'배치 경사 하강법(Batch Gradient Descent, Gradient Descent, BGD or GD)'**이라고 부릅니다. 반면, 미니 배치 단위로 경사 하강법을 수행하는 방법을 **'미니 배치 경사 하강법(Minibatch Gradient Descent)'**이라고 부릅니다. 그리고 배치 단위 1인 경우는 **'확률적 경사 하강법(Stochastic Gradient Descent, SGD)'**이라고 부릅니다.

* 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터를 사용하므로 가중치 값이 최적값에 수렴하는 과정이 매우 안정적이지만, 계산량이 너무 많이 듭니다. 미니 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터의 일부만을 보고 수행하므로 최적값으로 수렴하는 과정에서 값이 조금 헤매기도 하지만 훈련 속도가 빠릅니다.

* 배치 크기는 보통 2의 제곱수를 사용합니다. ex) 2, 4, 8, 16, 32, 64... 그 이유는 CPU와 GPU의 메모리가 2의 배수이므로 배치크기가 2의 제곱수일 경우에 데이터 송수신의 효율을 높일 수 있다고 합니다.

아래링크에서 BGD, Mini-batch GD, SGD 정의와 설명 및 예시를 볼 수 있습니다.

https://nonmeyet.tistory.com/entry/Batch-MiniBatch-Stochastic-%EC%A0%95%EC%9D%98%EC%99%80-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EC%98%88%EC%8B%9C

# 2. 이터레이션(Iteration)


미니 배치와 배치 크기의 정의에 대해서 이해하였다면 이터레이션(iteration)을 정의할 수 있습니다.

이터레이션은 한 번의 에포크 내에서 이루어지는 매개변수인 가중치 W와 b의 업데이트 횟수입니다. 전체 데이터가 2,000일 때 배치 크기를 200으로 한다면 이터레이션의 수는 총 10개입니다. 이는 한 번의 에포크 당 매개변수 업데이트가 10번 이루어짐을 의미합니다.

이제 미니 배치 학습을 할 수 있도록 도와주는 파이토치의 도구들을 알아봅시다.

# 3. 데이터 로드하기(Data Load)


파이토치에서는 데이터를 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 데이터셋(Dataset)과 데이터로더(DataLoader)를 제공합니다. 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있습니다. 기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것입니다.


Dataset을 커스텀하여 만들 수도 있지만 여기서는 텐서를 입력받아 Dataset의 형태로 변환해주는 TensorDataset을 사용해보겠습니다.


실습을 위해 기본적으로 필요한 파이토치의 도구들을 임포트합니다.



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

TensorDataset과 DataLoader를 임포트합니다.

In [ ]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

TensorDataset은 기본적으로 텐서를 입력으로 받습니다. 텐서 형태로 데이터를 정의합니다.

In [ ]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

이제 이를 TensorDataset의 입력으로 사용하고 dataset으로 저장합니다.

In [ ]:
dataset = TensorDataset(x_train, y_train)

파이토치의 데이터셋을 만들었다면 데이터로더를 사용 가능합니다. 데이터로더는 기본적으로 2개의 인자를 입력받는다. 하나는 데이터셋, 미니 배치의 크기입니다. 이때 미니 배치의 크기는 통상적으로 2의 배수를 사용합니다. (ex) 64, 128, 256...) 그리고 추가적으로 많이 사용되는 인자로 shuffle이 있습니다. shuffle=True를 선택하면 Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꿉니다.

사람도 같은 문제지를 계속 풀면 어느 순간 문제의 순서에 익숙해질 수 있습니다. 예를 들어 어떤 문제지의 12번 문제를 풀면서, '13번 문제가 뭔지는 기억은 안 나지만 어제 풀었던 기억으로 정답은 5번이었던 것 같은데' 하면서 문제 자체보단 순서에 익숙해질 수 있다는 것입니다. 그럴 때 문제지를 풀 때마다 문제 순서를 랜덤으로 바꾸면 도움이 될 겁니다. 마찬가지로 모델이 데이터셋의 순서에 익숙해지는 것을 방지하여 학습할 때는 이 옵션을 True를 주는 것을 권장합니다.

In [ ]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

이제 모델과 옵티마이저를 설계합니다.

In [ ]:
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

이제 훈련을 진행합니다. 아래 코드에서는 batch_idx와 samples를 주석 처리했는데 어떤 식으로 훈련되고 있는지 궁금하다면 주석 처리를 해제하고 훈련시켜보시기 바랍니다.

In [ ]:
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

nb_epochs = 0
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    
    print(batch_idx)
    print(samples)
 
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)
    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

In [ ]:
params = []
for param in model.parameters():
    params.append(param.view(-1))

print(params)

W_val = params[0].detach()
b_val = params[1].detach()

print(W_val)
print(b_val)

Cost의 값이 점차 작아집니다. (사실 아직 에포크를 더 늘려서 훈련하면 Cost의 값이 더 작아질 여지가 있습니다. 에포크를 늘려서도 훈련해보세요.) 이제 모델의 입력으로 임의의 값을 넣어 예측값을 확인합니다.

In [ ]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

# 4. 옵티마이저(Optimizer)

Optimizer는 딥러닝에서 Network가 빠르고 정확하게 학습하는 것을 목표로 한다. 주로 Gradient Descent Algorithm을 기반으로한 SGD에서 변형된 여러종류의 Optimizer가 사용되고 있다.
<figure>
<img src = "https://raw.githubusercontent.com/Hyun-chul/KAIA2022/main/Optimizer_history.png"/, height = 250, width = 500>
</figure>

참고링크: https://ganghee-lee.tistory.com/24




<figure>
<img src = "https://raw.githubusercontent.com/Hyun-chul/KAIA2022/main/Optimizer_graph1.gif"/, height = 250, width = 350>
</figure>


# 5. 과적합 (overfitting), 과소적합 (underfitting), 일반화 (genearlization)

- 과적합: 머신러닝 모델이 학습데이터에만 치중되어, 정작 테스트 데이터에서는 성능이 하락하는 경우

- 과소적합 (Underfitting): 학습데이터를 제대로 학습하지 못한 경우 

- 일반화(Generalization): 학습데이터와 학습되지 않은 실제 데이터 상에서 동시에 높은 성능을 내는 상태

<img src = "https://raw.githubusercontent.com/Hyun-chul/KAIA2022/main/classic%20overfitting_1.png">

출처: https://www.qfrjw.top/ProductDetail.aspx?iid=380054501&pr=39.88


## 과적합 발생원인?

- 학습할 샘플 데이터 수의 부족
    - 모집단(전체 집단, Population)의 데이터를 갖고 있고 이 데이터에 잘 맞는 모델을 만들었다면 전혀 문제가 없음
    - 하지만, 실제 우리가 예측하고자 하는 데이터는 표본(sample)집단으로 모집단에 포함되지 않음.
    - 즉, 학습할 데이터의 수가 적을수록 과적합 확률이 높음!

- 풀고자하는 문제에 비해 복잡한 모델 적용

    - 간단한 문제에 대해서는 간단한 모델 또는 적은 변수만을 사용하는 것이 과적합될 가능성을 줄일 수 있음. 모델이 복잡하다라는 표현은 많은 변수를 사용하는 것 또는 모델 자체가 복잡한 모델(SVM, 딥러닝 등)이라는 것을 의미

### 과적합을 방지하는 방법

- 데이터의 수를 늘려 모집단의 특성을 잘 반영한 데이터를 확보

- 풀고자하는 문제에 대하여 적절한 모형을 선택

- 적절한 실험설계

    - 데이터를 학습/검증/테스트 데이터로 랜덤하게 분할

    - 학습 데이터(Training Data) : 머신러닝 모델 함수𝑓를 적합 시킬 데이터

    - 검증 데이터(Validation Data) : 학습데이터를 적합 시킨 함수𝑓를 검증 시킬 데이터

    - 테스트데이터(Test Data) :테스트데이터는 최종적으로 우리가 만든 모델의 성능을 측정하는 데이터임

    - 검증데이터와 테스트데이터의 다른점은, 테스트데이터는 우리가 전혀 보지못한 데이터라고 가정 해야 한다라는 것
    


<figure>
<img src = "https://raw.githubusercontent.com/Hyun-chul/KAIA2022/main/Overfitting.png"/, height = 230, width = 500>
</figure>

예시: PyTorch K-Fold Cross-Validation using Dataloader and Sklearn

https://androidkt.com/pytorch-k-fold-cross-validation-using-dataloader-and-sklearn/
